In [ ]:
from dotenv import load_dotenv
import os
import base64
from requests import post,get
import json
import requests

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists")
        return None
    
    return json_result[0]

def get_songs_recommendations(token, artist_id, min_popularity, max_popularity):
    url = f"https://api.spotify.com/v1/recommendations?limit=100&market=US&seed_artists={artist_id}&min_popularity={min_popularity}&max_popularity={max_popularity}"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    if result.status_code ==200:
        json_result=json.loads(result.content)
        return json_result['tracks']
    else:
        print(f"Failed to fetch recommendations. Status code: {result.status_code}")
        return None


In [ ]:
import pandas as pd
artist_df = pd.read_csv('top_artist.csv', index_col= 0)

In [ ]:
artist_df

In [ ]:
artists = artist_df['Artist']

In [ ]:
token = get_token()
artist_id_list = []
for artist in artists:
    artist_id = search_for_artist(token, artist)
    artist_id_list.append(artist_id['id'])

artist_id_list

In [ ]:
artist_dict = {
    'artist':artists,
    'artist_id':artist_id_list
}
artist_id_df = pd.DataFrame(artist_dict)
artist_id_df

In [ ]:
artist_id_df.to_csv('artist_id.csv')